In [ ]:
library(Seurat)
library(stringr)
library(data.table)
library(tidyverse)
library(clusterProfiler)
library(org.Hs.eg.db)
library(ggplot2)
library(cowplot)
library(SingleR)
library(xlsx)
library(writexl)
library(dplyr)
library(ggrepel)
library(svglite)
library(Tempora)
library(ggalluvial)
library(RColorBrewer)
library(ggpubr)

library(Tempora)
library(Seurat)
library(RCurl)
library(tidyverse)
library(igraph)
library(ggraph)
library(graphlayouts)
library(ggforce)
library(scatterpie)
library(RColorBrewer)
library(igraph)
library(ggrepel)
library(stringr)
library(scales)
library(patchwork)


In [ ]:
save_plot = function(plotobj,outname, fig.width, fig.height)
{
  print(paste(outname, fig.width, fig.height))
      
  fname=paste(outname, "png", sep=".")
  print(paste("Saving to file", fname))
  png(filename=fname, width = fig.width, height = fig.height, units = 'in', res = 300)#width = fig.width*100, height=fig.height*100)
  plot(plotobj)
  dev.off()
  
  fname=paste(outname, "pdf", sep=".")
  print(paste("Saving to file", fname))
  pdf(file=fname, width = fig.width, height=fig.height)
  plot(plotobj)
  dev.off()
  

  fname=paste(outname, "svg", sep=".")
  print(paste("Saving to file", fname))
  svglite::svglite(file = fname, width = fig.width, height = fig.height)
  plot(plotobj)
  dev.off()
  
  return(plotobj)
}


In [ ]:
indf = read.table("all_umi_counts.tsv", header=TRUE, sep ="\t")

In [ ]:
rownames(indf) = indf$gene_name

In [ ]:
indf$gene_name = NULL

In [ ]:
makeSeuratObj = function(matrix, proj, minUMIs, plots)
{
    obj = CreateSeuratObject(matrix, project=proj)
    print("Renaming Cells")
    obj <- RenameCells(obj, add.cell.id=proj)
    
    print(paste("Seurat obj project", obj@project.name))
    
    #obj[["percent.mtrp"]] <- PercentageFeatureSet(obj, pattern = "^mt-|^Rps|^Rpl")
    
    mtPattern = "^MT-"
    rplPattern = "^RPL"
    rpsPattern = "^RPS"
    rpPattern = "^RPS|^RPL"
    
    selGenes = rownames(obj)[grepl(rownames(obj), pattern=mtPattern)]
    print(paste("Got a total of mt-Genes:", length(selGenes), paste0(head(unlist(selGenes)), collapse=", ")))
    
    selGenes = rownames(obj)[grepl(rownames(obj), pattern=rplPattern)]
    print(paste("Got a total of Rpl-Genes:", length(selGenes), paste0(head(unlist(selGenes)), collapse=", ")))
    
    selGenes = rownames(obj)[grepl(rownames(obj), pattern=rpsPattern)]
    print(paste("Got a total of Rps-Genes:", length(selGenes), paste0(head(unlist(selGenes)), collapse=", ")))
    
    selGenes = rownames(obj)[grepl(rownames(obj), pattern=rpPattern)]
    print(paste("Got a total of Rp-Genes:", length(selGenes), paste0(head(unlist(selGenes)), collapse=", ")))
    
    obj[["percent.mt"]] <- PercentageFeatureSet(obj, pattern = mtPattern)
    obj[["percent.rpl"]] <- PercentageFeatureSet(obj, pattern = rplPattern)
    obj[["percent.rps"]] <- PercentageFeatureSet(obj, pattern = rpsPattern)
    obj[["percent.rp"]] <- PercentageFeatureSet(obj, pattern = rpPattern)
    
    if (plots)
    {
      plot1 <- FeatureScatter(obj, feature1 = "nCount_RNA", feature2 = "percent.mt")
      plot2 <- FeatureScatter(obj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
      show(plot1 + scale_x_continuous(n.breaks = 20) + scale_y_continuous(n.breaks = 20))

    }
    

    # mt content: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6072887/
    obj <- subset(obj, subset = nFeature_RNA > 200 & nFeature_RNA < 40000 & nCount_RNA > minUMIs & percent.mt < 15 & percent.rp < 40)
    
    return(obj)
}

In [ ]:
obj.bulk = makeSeuratObj(indf, "bulk", 1000, TRUE)

In [ ]:
print(dim(indf))

In [ ]:
obj.bulk

In [ ]:
VlnPlot(obj.bulk, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3, pt.size = 0.001, group.by = "orig.ident")

In [ ]:
obj.bulk <- NormalizeData(obj.bulk, normalization.method = "LogNormalize", scale.factor = 10000)
obj.bulk <- FindVariableFeatures(obj.bulk, nfeatures = 2000)

In [ ]:
obj.bulk <- ScaleData(obj.bulk, verbose = FALSE)
obj.bulk <- RunPCA(obj.bulk, npcs = 30, verbose = FALSE)

In [ ]:
obj.bulk <- RunUMAP(obj.bulk, reduction = "pca", dims = 1:30)
obj.bulk <- FindNeighbors(obj.bulk, reduction = "pca", dims = 1:30)
obj.bulk <- FindClusters(obj.bulk, resolution = 0.5)

In [ ]:
DimPlot(obj.bulk, reduction="umap", group.by="orig.ident")

In [ ]:
saveRDS(obj.bulk, "bulk_seurat_obj.rds")

In [ ]:
obj.bulk=readRDS("bulk_seurat_obj.rds")

In [ ]:
cellnames = names(obj.bulk$orig.ident)

mpoint = unlist(lapply(str_split(cellnames, "_"), function(x){return(x[3])}))
mpoint[mpoint == "tag4nl"] = "tag4"

mpoint_fact = factor(as.factor(mpoint), levels=c('ctrl','tag4', 'corkum','tag11','tag60'))


mpoint[mpoint == "tag4"] = "Day4"
mpoint[mpoint == "tag11"] = "Day11"
mpoint[mpoint == "tag60"] = "Day60"
mpoint[mpoint == "ctrl"] = "Ctrl"
mpoint[mpoint == "corkum"] = "Hosp."

mpoint2_fact = factor(as.factor(mpoint), levels=c('Ctrl','Day4', 'Hosp.','Day11','Day60'))

mpointcolors = as.vector(mpoint)
mpointcolors[mpointcolors == "Ctrl"] = "#1F1F1F"
mpointcolors[mpointcolors == "Hosp."] = "#E93A8D"
mpointcolors[mpointcolors == "Day4"] = "#069038"
mpointcolors[mpointcolors == "Day11"] = "#5DE223"
mpointcolors[mpointcolors == "Day60"] = "#CCCFCF"

mpoint2Colors = c("#1F1F1F", "#069038","#E93A8D","#5DE223","#CCCFCF")

obj.bulk$colors = mpointcolors

obj.bulk$mpoint = mpoint_fact
obj.bulk$mpoint2 = mpoint2_fact
obj.bulk$celltype = obj.bulk$orig.ident
obj.bulk$ct_time = paste(obj.bulk$celltype, obj.bulk$mpoint2)

In [ ]:
cells.control = names(obj.bulk$mpoint[obj.bulk$mpoint == "ctrl"])
cells.tag4 = names(obj.bulk$mpoint[obj.bulk$mpoint == "tag4"])
cells.tag11 = names(obj.bulk$mpoint[obj.bulk$mpoint == "tag11"])
cells.tag60 = names(obj.bulk$mpoint[obj.bulk$mpoint == "tag60"])
cells.corkum = names(obj.bulk$mpoint[obj.bulk$mpoint == "corkum"])


cells.symptomatic = names(obj.bulk$mpoint)

In [ ]:
DimPlot(obj.bulk, reduction="pca", group.by="orig.ident")

In [ ]:
p=DimPlot(obj.bulk, reduction="pca", group.by="mpoint")
save_plot(p, "detests_time/pca_mpoint", 12, 12)

In [ ]:
p=DimPlot(obj.bulk, reduction="umap", group.by="mpoint")
save_plot(p, "detests_time/umap_mpoint", 12, 12)

In [ ]:
interferonGenes = c("MT2A", "ISG15", "LY6E", "IFIT1", "IFIT2", "IFIT3", "IFITM1", "IFITM3", "IFI44L", "IFI6", "MX1", "IFI27",  "IFI44L", "RSAD2", "SIGLEC1", "IFIT1", "ISG15")
print(length(interferonGenes))

In [ ]:
makesummary = function(a, suffix)
{
  out = {}
  out["num"] = length(a)
  
  if (length(a) == 0)
  {
    f = c(0,0,0,0,0)
    meanA = 0
  } else {
    f = fivenum(a)
    meanA = mean(a)
  }

  out["min"] = f[1]
  out["lower_hinge"] = f[2]
  out["median"] = f[3]
  out["upper_hinge"] = f[4]
  out["max"] = f[5]
  out["mean"] = meanA
  
  names(out) = paste(names(out), suffix, sep=".")
  
  return(out)
}

getExprData = function(markerObj, markerCells, sampleSuffix, assay="RNA")
{
  expTable = GetAssayData(object = subset(x=markerObj, cells=markerCells), slot = "data", assay=assay)
  allgenes = rownames(expTable)
  cellnames = colnames(expTable)

  expt.r = as(expTable, "dgTMatrix")
  expt.df = data.frame(r = expt.r@i + 1, c = expt.r@j + 1, x = expt.r@x)

  DT <- data.table(expt.df)
  res = DT[, as.list(makesummary(x, sampleSuffix)), by = r]
  res[[paste("anum", sampleSuffix, sep=".")]] = length(cellnames)
  res$gene = allgenes[res$r]
  
  res = res[,r:=NULL]
  
  return(res)
}

getDEXpressionDF = function ( scdata, markers, assay="SCT" )
{

outDF = NULL
DefaultAssay(object=scdata) = assay  
clusterIDs = as.character(sort(unique(Idents(scdata))))

scCells = Idents(scdata)
scCells = names(scCells)
scCells = unlist(as.character(scCells))

for (clusterID in clusterIDs){
    
    print(clusterID)
    
    cellIdents = Idents(scdata)
    cellIdents.c = names(cellIdents[cellIdents == clusterID])
    cellIdents.c = unlist(lapply(cellIdents.c, as.character))  
    
    cellIdents.bg = setdiff(unlist(lapply(names(cellIdents), as.character)), cellIdents.c)
    
    expvals = getExprData(scdata, cellIdents.c, "cluster", assay=assay)
    expvals.bg = getExprData(scdata, cellIdents.bg, "bg", assay=assay)

    modmarkers = markers[[clusterID]]
    modmarkers$gene = rownames(modmarkers)
    
    markerdf = as.data.frame(modmarkers)
    
    if ((nrow(markerdf) > 0) && (nrow(expvals) > 0))
    {
      expvals = merge(markerdf, expvals, all.x=T, by.x="gene", by.y = "gene")  
    }
    
    if ((nrow(expvals) > 0) && (nrow(expvals.bg) > 0))
    {
      expvals = merge(expvals, expvals.bg, all.x=T, by.x="gene", by.y = "gene")  
    }
    
    expvals = as.data.frame(cbind(clusterID, expvals))
    
    if (!is.data.frame(outDF) || nrow(outDF)==0)
    {
    outDF = expvals
    } else {
    outDF = as.data.frame(rbind(outDF, expvals))
    }
    
}

return(outDF)

}

makeDEResults = function(inobj, assay="SCT", test="wilcox")
{
  clusterIDs = as.character(sort(unique(Idents(inobj))))
  
  retList = list()
  
  for(clusterID in clusterIDs)
  {
  
  
      cellIdents = Idents(inobj)
      cellIdents.c = names(cellIdents[cellIdents == clusterID])
      cellIdents.c = unlist(lapply(cellIdents.c, as.character))
  
      print(paste("Processing cluster", clusterID, "with a total of", length(cellIdents.c), "cells"))
  
      deMarkers = FindMarkers(inobj, assay=assay, ident.1 = cellIdents.c, test.use=test)
  
  
      retList[[clusterID]] = deMarkers
  
  }
  
  return(retList)

}


compareCells = function(scdata, cellsID1, cellsID2, suffix1, suffix2, prefix="cluster", test="t", assay="RNA", outfolder="./", all=FALSE)
{
    logfc.threshold = 0.25
    
    if (all==TRUE)
    {
    logfc.threshold = 0.01  
    }
    
    print(paste("Missing cells", cellsID1[!cellsID1 %in% colnames(scdata)]))
    print(paste("Missing cells", cellsID2[!cellsID2 %in% colnames(scdata)]))
    
    print(paste("Selected Cells 1", length(cellsID1)))
    print(paste("Selected Cells 2", length(cellsID2)))
    
    if ((length(cellsID1) < 3) || (length(cellsID2) < 3))
    {
      return(data.frame())
    }

    markers = FindMarkers(scdata, assay=assay, ident.1 = cellsID1, ident.2 = cellsID2, test.use=test, logfc.threshold=logfc.threshold)
    
    outvalues1 = getExprData(scdata, cellsID1, suffix1, assay=assay)
    outvalues2 = getExprData(scdata, cellsID2, suffix2, assay=assay) 
    
    
    markers$gene = rownames(markers)
    joinedData = merge(markers, outvalues1, by="gene", all=T)
    joinedData = merge(joinedData, outvalues2, by="gene", all=T)  
    
    joinedData = joinedData[!is.na(joinedData$p_val),]
    
    outfile = paste(outfolder, "/", prefix, ".", suffix1, "_", suffix2, ".tsv", sep="")
    message(outfile)
    write.table(joinedData, file=outfile, row.names = F,  quote=FALSE, sep='\t')
    
    outfile = paste(outfolder, "/", prefix, ".", suffix1, "_", suffix2, ".xlsx", sep="")
    message(outfile)
    write.xlsx(joinedData, file=outfile, row.names = F)
    
    return(joinedData)
}



In [ ]:


geneExpr = list()

allCellTypes = list(
  "Monocytes" = names(obj.bulk$orig.ident[obj.bulk$orig.ident=="Monocytes"]),
  "NK" = names(obj.bulk$orig.ident[obj.bulk$orig.ident=="NK"]),
  "CD4" = names(obj.bulk$orig.ident[obj.bulk$orig.ident=="CD4"]),
  "CD8" = names(obj.bulk$orig.ident[obj.bulk$orig.ident=="CD8"]),
  "All"=names(obj.bulk$orig.ident)
)

for (cellSubsetName in names(allCellTypes))
{
    print(cellSubsetName)
    
    cellsubset = allCellTypes[[cellSubsetName]]
    
    exprByTP = list()

    for (mp2 in unique(obj.bulk$mpoint2))
    {
        print(mp2)
        selCells = intersect(names(obj.bulk$mpoint2[obj.bulk$mpoint2 == mp2]), cellsubset)
        print(length(selCells))
        outvalues1 = getExprData(obj.bulk, selCells, mp2, assay="RNA")
                
        exprByTP[[mp2]] = outvalues1
    }
    
    geneExpr[[cellSubsetName]] = exprByTP
        
}



In [ ]:
prepareFuzzyDataExpr = function(deObj, celltype, use.quantiles=c(0.25, 0.75))
{
    
    deDFs = deObj[[celltype]]
    

    celltype.logfcs = c()
    short.results = list()

    my_merge_outer <- function(df1, df2){                                # Create own merging function
      merge(df1, df2, by = "gene", all=TRUE)
    }

    for (condtp in names(deDFs))
    {
      #print(colnames(deDFs[[condtp]]))
      colname = paste("mean", condtp, sep=".")
      #print(paste(condtp, colname))
      celltype.logfcs = c(celltype.logfcs, as.vector(deDFs[[condtp]][[colname]]))
    }
    
    celltype.logfcs = celltype.logfcs[celltype.logfcs>0]


    createBins = function(indf, col.fc, col.sig, bounds)
    {

      smalldf = indf[, c("gene", col.fc, col.sig)]
      colnames(smalldf) = c("gene", "fc", "sig")

      binFun = function(x)
      {

        fc = as.numeric(x[2])
        sig = as.numeric(x[3])

        #print(x)
        #print(bounds)

        if ((is.na(sig)) || is.nan(sig) ||(sig > 0.05) || is.na(fc) || is.nan(fc))
        {
          return(0)
        }

        curBin = 0

        if (fc > bounds[length(bounds)])
        {
          return(length(bounds))
        }

        for (i in 1:length(bounds))
        {
          #print(paste(i, curBin+i-1, fc, "<", bounds[i], fc < bounds[i]))
          if (fc < bounds[i])
          {
            return(curBin + i -1)
          }
        }

        print(x)
        return(10)
      }


      bins = apply(smalldf, 1, binFun)

      return(bins)

    }

    #testDF = data.frame(gene=c("gene1", "gene2", "gene3", "gene4", "gene5"),
    #                    log2FC = c(-1, -0.55, 0.1, 0.55, 1),
    #                    p_val_adj = c(0.01, 0.01, 0.01, 0.01,0.01)
    #)
    #createBins(testDF, "log2FC", "p_val_adj", c(-0.75, -0.5, 0.5, 0.75))


    quantileVec = as.numeric(quantile(abs(celltype.logfcs), probs = use.quantiles, na.rm=T))
    # using expression values!#quantileVec = c(-rev(quantileVec), quantileVec)
    print(paste("Quantile Vector"))
    print(quantileVec)



      condtp.results = list()
      for (tp in names(deDFs))
      {

          celltype.logfcs = c(celltype.logfcs, deDFs[[condtp]][[tp]]$avg_log2FC)

          resDF = data.frame(gene=deDFs[[tp]]$gene)
          
          colname = paste("mean", tp, sep=".")
          resDF[[paste("avg_log2FC", tp, sep=".")]] = as.vector(deDFs[[tp]][[colname]])
          resDF[[paste("p_val_adj", tp, sep=".")]] = 0
          resDF[[paste("bins", tp, sep=".")]] = bins = createBins(resDF, paste("avg_log2FC", tp, sep="."),
                                                                          paste("p_val_adj", tp, sep="."), quantileVec)

          condtp.results[[tp]] = resDF
      }

    celltype.results = Reduce(my_merge_outer, condtp.results) 



    for (binID in grep("^bins.", colnames(celltype.results), value = T))
    {
      celltype.results[[binID]][is.na(celltype.results[[binID]])] = 0

    }

    return(celltype.results)
    
}

In [ ]:
  ctrl.asymp.tag4 = compareCells(obj.integrated,
                  intersect(cells.control, cells.celltype),
                  intersect(intersect(cells.asymptomatic, cells.tag4), cells.celltype),
                  "control", "asymptomatic", prefix=paste(celltype, "tag4", sep="."), test="t", outfolder="detests_time")
  ctrl.asymp.tag11 = compareCells(obj.integrated,
                  intersect(cells.control, cells.celltype),
                  intersect(intersect(cells.asymptomatic, cells.tag11), cells.celltype),
                  "control", "asymptomatic", prefix=paste(celltype, "tag11", sep="."), test="t", outfolder="detests_time")
  ctrl.asymp.tag60 = compareCells(obj.integrated,
                  intersect(cells.control, cells.celltype),
                  intersect(intersect(cells.asymptomatic, cells.tag60), cells.celltype),
                  "control", "asymptomatic", prefix=paste(celltype, "tag60", sep="."), test="t", outfolder="detests_time")
  listAsympt = list("tag4"=ctrl.asymp.tp1, "tag11"=ctrl.asymp.tp2, "tag60"=ctrl.asymp.tp3)


In [ ]:
allCellTypes = list(
  "Monocytes" = names(obj.bulk$orig.ident[obj.bulk$orig.ident=="Monocytes"]),
  "NK" = names(obj.bulk$orig.ident[obj.bulk$orig.ident=="NK"]),
  "CD4" = names(obj.bulk$orig.ident[obj.bulk$orig.ident=="CD4"]),
  "CD8" = names(obj.bulk$orig.ident[obj.bulk$orig.ident=="CD8"]),
  "all"=names(obj.bulk$orig.ident)
)

lapply(allCellTypes, length)
allResults=list()
for (celltype in names(allCellTypes))
{
  
  cells.celltype = allCellTypes[[celltype]]
  
  ctrl.symp.corkum = compareCells(obj.bulk,
                  intersect(cells.control, cells.celltype),
                  intersect(intersect(cells.symptomatic, cells.corkum), cells.celltype),
                  "control", "asymptomatic", prefix=paste(celltype, "corkum", sep="."), test="t", outfolder="detests_time")
  ctrl.symp.tag4 = compareCells(obj.bulk,
                  intersect(cells.control, cells.celltype),
                  intersect(intersect(cells.symptomatic, cells.tag4), cells.celltype),
                  "control", "asymptomatic", prefix=paste(celltype, "tag4", sep="."), test="t", outfolder="detests_time")
  ctrl.symp.tag11 = compareCells(obj.bulk,
                  intersect(cells.control, cells.celltype),
                  intersect(intersect(cells.symptomatic, cells.tag11), cells.celltype),
                  "control", "asymptomatic", prefix=paste(celltype, "tag11", sep="."), test="t", outfolder="detests_time")
  ctrl.symp.tag60 = compareCells(obj.bulk,
                  intersect(cells.control, cells.celltype),
                  intersect(intersect(cells.symptomatic, cells.tag60), cells.celltype),
                  "control", "asymptomatic", prefix=paste(celltype, "tag60", sep="."), test="t", outfolder="detests_time")
  
  listSympt =  list("corkum"=ctrl.symp.corkum, "tag4"=ctrl.symp.tag4,  "tag11"=ctrl.symp.tag11,  "tag60"=ctrl.symp.tag60)
  
  allResults[[celltype]] = list("asympt"=listSympt)#, "asympt"=listAsympt)
  
}



In [ ]:
saveRDS(allResults, "detests_time/allDEResults.rds")

In [ ]:
allResults = readRDS("detests_time/allDEResults.rds")

In [ ]:
PlotVaryingPWs <- function(object, identifier, outprefix, pw.max=50){

  if (class(object)[1] != "Tempora"){
    stop("Not a valid Tempora object")
  }
  if (is.null(object@varying.pws)){
    print("IdentifyVaryingPWs has not been run or no temporally varying pathways were detected. Please run IdentifyVaryingPWs or re-run with a more relaxed p-value cutoff See ?Tempora::IdentifyVaryingPWs for details")
    return()
  }

  varying_pathways <- sort(object@varying.pws)

  if (length(varying_pathways) > pw.max)
  {
    varying_pathways = varying_pathways[1:px.max]
  }
  gsva_bycluster <- object@cluster.pathways
  gams <- object@gams
    
    annotDF = object@cluster.metadata[, c("Id", "label")]
    annotDF$chrID = as.character(annotDF$Id)
    clusterLabels = annotDF$label
    clusterLabels = unlist(lapply(str_split(annotDF$label, "-"), function(x){x[2]}))

    names(clusterLabels) = annotDF$chrID


  cat("\nPlotting time-dependent pathways...")
  print(paste("Number of time-dependent pathways", length(varying_pathways)))

  if (!dir.exists(outprefix))
  {
    dir.create(outprefix)
  }

  for (i in 1:length(varying_pathways)){
    fname=paste(outprefix, paste("PlotVaryingPWs", identifier, i, sep="_"), sep="/")
    print(paste("Saving to file", fname))
    plot.new()
    png(filename=paste(fname, "png", sep="."), width = 8, height = 6, units = 'in', res = 300)#width = fig.width*100, height=fig.height*100)

      par(mar=c(5,5,5, 10))      
        
      if (length(grep(names(varying_pathways)[i], rownames(gsva_bycluster))) > 1){
        plot_df <- data.frame(cluster=colnames(gsva_bycluster[grep(names(varying_pathways)[i], rownames(gsva_bycluster)), ]), value=colMeans(gsva_bycluster[grep(names(varying_pathways)[i], rownames(gsva_bycluster)), ]))
        plot_df$time <- object@cluster.metadata$Cluster_time_score
        plot_df$cluster = clusterLabels[plot_df$cluster]
      }
      else if (length(grep(names(varying_pathways)[i], rownames(gsva_bycluster))) == 1) {
        plot_df <- data.frame(cluster=names(gsva_bycluster[grep(names(varying_pathways)[i], rownames(gsva_bycluster)), ]), value=gsva_bycluster[grep(names(varying_pathways)[i], rownames(gsva_bycluster)), ])
        plot_df$time <- object@cluster.metadata$Cluster_time_score
        plot_df$cluster = clusterLabels[plot_df$cluster]
      }
      id <- which(names(gams)==names(varying_pathways)[i])
      mgcv::plot.gam(gams[[id[1]]], main = paste0(names(varying_pathways)[i]), xlab = "Inferred time", ylab="Pathway expression level", bty="l", cex.main = 1, xaxt = "n", shade= F, se=3, scheme=1)
      xmin <- par("usr")[1]
      xmax <- par("usr")[2]
      points(x=plot_df$time, y=plot_df$value, pch=20, col="navy", cex=0.9, xpd=NA)
      text(x=plot_df$time, y=plot_df$value, labels=plot_df[,1], pos = 4, cex = 0.5, col="navy", xpd=NA)
      legend("topright", legend = "Cluster", pch = 20, col = "navy", bty="n", text.col="navy", cex=0.9)
      legend("topright", legend=paste0("\nAdjusted p-value = ", round(varying_pathways[[i]], 5)), bty="n", cex=0.9)
      axis(side=1, at=c(xmin, xmax), labels = c("Early", "Late"), tick=T)

    dev.off()


    
    print(paste("Saving to file", fname))
    pdf(file=paste(fname, "pdf", sep="."),width = 8, height = 6)

      par(mar=c(5,5,5, 10))      
        
      if (length(grep(names(varying_pathways)[i], rownames(gsva_bycluster))) > 1){
        plot_df <- data.frame(cluster=colnames(gsva_bycluster[grep(names(varying_pathways)[i], rownames(gsva_bycluster)), ]), value=colMeans(gsva_bycluster[grep(names(varying_pathways)[i], rownames(gsva_bycluster)), ]))
        plot_df$time <- object@cluster.metadata$Cluster_time_score
        plot_df$cluster = clusterLabels[plot_df$cluster]
      }
      else if (length(grep(names(varying_pathways)[i], rownames(gsva_bycluster))) == 1) {
        plot_df <- data.frame(cluster=names(gsva_bycluster[grep(names(varying_pathways)[i], rownames(gsva_bycluster)), ]), value=gsva_bycluster[grep(names(varying_pathways)[i], rownames(gsva_bycluster)), ])
        plot_df$time <- object@cluster.metadata$Cluster_time_score
        plot_df$cluster = clusterLabels[plot_df$cluster]
      }
      id <- which(names(gams)==names(varying_pathways)[i])
      mgcv::plot.gam(gams[[id[1]]], main = paste0(names(varying_pathways)[i]), xlab = "Inferred time", ylab="Pathway expression level", bty="l", cex.main = 1, xaxt = "n", shade= F, se=3, scheme=1)
      xmin <- par("usr")[1]
      xmax <- par("usr")[2]
      points(x=plot_df$time, y=plot_df$value, pch=20, col="navy", cex=0.9, xpd=NA)
      text(x=plot_df$time, y=plot_df$value, labels=plot_df[,1], pos = 4, cex = 0.5, col="navy", xpd=NA)
      legend("topright", legend = "Cluster", pch = 20, col = "navy", bty="n", text.col="navy", cex=0.9)
      legend("topright", legend=paste0("\nAdjusted p-value = ", round(varying_pathways[[i]], 5)), bty="n", cex=0.9)
      axis(side=1, at=c(xmin, xmax), labels = c("Early", "Late"), tick=T)


    dev.off()
      
  }

}                         
                         
IdentifyVaryingPWs <- function(object, pval_threshold=0.05){

  if (class(object)[1] != "Tempora"){
    stop("Not a valid Tempora object")
  }
  if (is.null(object@n.pcs)){
    stop("BuildTrajectory has not been run. See ?Tempora::BuildTrajectory for details")
  }
  if (is.null(object@cluster.pathways)){
    stop("CalculatePWProfiles has not been run. See ?Tempora::CalculatePWProfiles for details")
  }
  gsva_bycluster <- object@cluster.pathways

  significant_pathways <- c()
  for (i in 1:object@n.pcs){
    genes_scaled <- scale(object@cluster.pathways.dr$rotation[,i])
    significant_pathways <- c(names(which(genes_scaled[,1] > 1.5 | genes_scaled[,1] < -1.5)), significant_pathways)
  }

  pca_pathways <- sub("%.*", "", significant_pathways)
  pca_pathways <- gsub("\\s*\\([^\\)]+\\)","",pca_pathways)
  pca_pathways_cleaned <- gsub("[[:punct:]]", "", pca_pathways)
  themes <- pca_pathways_cleaned

  cat("Fitting GAM models...")

  p_vals <- gams <- list()
  for (i in 1:length(themes)){
      if (i %% 1000 == 0)
      {
        print(paste(i, "of", length(themes)))
      }
    
    if(length(grep(themes[i], rownames(gsva_bycluster))) == 0) {
      p_vals[[i]] <- 1
      gams[[i]] <- NA
      next
      }
    if (length(grep(themes[i], rownames(gsva_bycluster))) > 1){
      plot_df <- data.frame(cluster=colnames(gsva_bycluster[grep(themes[i], rownames(gsva_bycluster)), ]), value=colMeans(gsva_bycluster[grep(themes[i], rownames(gsva_bycluster)), ], na.rm=T))
    } else if (length(grep(themes[i], rownames(gsva_bycluster))) == 1){
      plot_df <- data.frame(cluster=names(gsva_bycluster[grep(themes[i], rownames(gsva_bycluster)), ]), value=gsva_bycluster[grep(themes[i], rownames(gsva_bycluster)), ]) }
    plot_df$time <- object@cluster.metadata$Cluster_time_score
    gams[[i]] <- mgcv::gam(value ~ s(time, k=3, bs='cr'), data=plot_df)
    temp_anova <- mgcv::anova.gam(gams[[i]])
    p_vals[[i]] <- temp_anova$s.pv
  }

  names(p_vals) <- names(gams) <- themes

  pval_threshold = pval_threshold
  p_vals_adj <- p.adjust(unlist(p_vals[which(unlist(p_vals) > 0)]), method = "BH")
  varying_pathways <- p_vals_adj[which(p_vals_adj < pval_threshold)]
  varying_pathways <- varying_pathways[!duplicated(names(varying_pathways))]

  if (length(varying_pathways)==0){
    cat("No temporally varying pathways detected. Please try running IdentifyVaryingPWs with a more relaxed p-value cutoff.")
    #eventhough the function was not successful return the object because in the vignette
    # this function call sets the original object to what is returned and if it is null
    # you loose all the processing you have done until now. 
    return(object)
  } else {
    object@varying.pws <- varying_pathways
    object@gams <- gams
    return(object)
  }
}


makeTrajectoryPlot = function(covid_tempora, colors=NULL)
{
  edge_graph <- igraph::graph_from_data_frame(d=covid_tempora@trajectory, vertices = covid_tempora@cluster.metadata, directed = T) 
l <- igraph::layout_with_sugiyama(edge_graph, hgap=50, vgap=100, layers = covid_tempora@cluster.metadata$Cluster_time_score, maxiter = 2000)
colours <- brewer.pal(length(levels(covid_tempora@meta.data$Timepoints)), "YlOrRd")

g = edge_graph
# precompute layout
V(g)$x <- l$layout[,1]
V(g)$y <- l$layout[,2]
    
    
                     
l$layout[,1] = 500*(l$layout[,1]/max(l$layout[,1]))

xMinO = round(min(l$layout[,1]))
xMaxO = round(max(l$layout[,1]))

xMin = xMinO - max(c(0.05*(xMaxO-xMinO), 100))
xMax = xMaxO + max(c(0.05*(xMaxO-xMinO), 100))

yMinO = round(min(l$layout[,2]))
yMaxO = round(max(l$layout[,2]))

yMin = yMinO - max(c(0.05*(yMaxO-yMinO), 100))
yMax = yMaxO + max(c(0.05*(yMaxO-yMinO), 100))
                         
V(g)$x <- l$layout[,1]
V(g)$y <- l$layout[,2]
    
V(g)$label=unlist(lapply(str_split(covid_tempora@cluster.metadata$label, "-"), function(x){x[2]}))
aesDF=data.frame("x"=V(g)$x, "y"=V(g)$y)
for (tpx in unique(covid_tempora@meta.data$Timepoints))
{
  tpData = unlist(lapply(1:nrow(covid_tempora@cluster.metadata), function(x) as.numeric(covid_tempora@cluster.metadata[x,2:((length(levels(covid_tempora@meta.data$Timepoints)))+1)][tpx])))
  aesDF[[as.character(tpx)]] = tpData
}
aesDF$radius = 25

print(l)

print(paste(xMin, xMax, yMin, yMax))

if (is.null(colors))
{
getPalette = colorRampPalette(brewer.pal(10, "Set3"))
nTimepoints = length(unique(covid_tempora@meta.data$Timepoints))
colors=getPalette(nTimepoints)
}                         


aesCols = as.character(levels(unique(covid_tempora@meta.data$Timepoints)))

#geom_scatterpie(cols = aesCols, data = aesDF, colour = "white", pie_scale = 0.5) +
                                                  
p=ggraph(g, "manual", x = V(g)$x, y = V(g)$y) +
  geom_edge_link(arrow = arrow(length = unit(5, 'mm')), end_cap = circle(7, 'mm')) + xlim(xMin, xMax)+geom_scatterpie(aes(x=x, y=y, r=radius), cols = aesCols, data = aesDF, colour = "white") +
  geom_text_repel(aes(x=x, y=y, label=label), point.size=20, size=5, max.overlaps=10)+
  scale_fill_manual(name="Timepoint", values = colors) +
  coord_fixed() + 
  scale_y_continuous(expand = c(0, 0),limits = c(yMin, yMax), breaks = c(yMin,yMax), minor_breaks = NULL, label=c("Late", "Early"))+ ylab("Inferred Time")+
  theme_minimal() +  theme(
    legend.position = "bottom",
    axis.text.x = element_blank(),
    axis.ticks.x = element_blank(),
    axis.title.x=element_blank(),
    axis.text.y = element_text(face="bold", size=18),
    axis.title.y = element_text(face="bold", size=24),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    axis.line.y = element_line(color='black'),
    legend.text=element_text(size=24),
    legend.title = element_text(size=24)
    )

return(p)
}


analyseTempora = function( scobj, clusterField, clusterFieldLevels, timepointField, timepoints, outprefix, pw.threshold=0.05, pw.max=50, pw.genes.max=500,pw.genes.min=10,do.pw=TRUE )
{
  covid_tempora <- ImportSeuratObject(scobj,
                                     clusters = clusterField,
                                     timepoints = timepointField, 
                                     assayType = "RNA",
                                     cluster_labels=clusterFieldLevels,
                                     timepoint_order = timepoints)

gmt_file = "detests_time/Human_COVID.gmt"


covid_tempora <- CalculatePWProfiles(covid_tempora, 
                gmt_path = gmt_file,
                method="gsva", min.sz = pw.genes.min, max.sz = pw.genes.max, parallel.sz = 1)
n_calc_pcs = ncol(covid_tempora@cluster.pathways.dr$x)
print(paste("Calculated number of PCs:", n_calc_pcs))


print("Building Trajectory")
covid_tempora = Tempora::BuildTrajectory(covid_tempora, n_pcs = n_calc_pcs, difference_threshold = 0.01)



print("Drawing Trajectory")

p = makeTrajectoryPlot(covid_tempora)
p=save_plot(p, paste(outprefix, "trajectory", sep="_"),  fig.width = 15, fig.height = 5)

if (do.pw)
{
print("IdentifyVaryingPWs")
#Fit GAMs on pathway enrichment profile
covid_tempora <- IdentifyVaryingPWs(covid_tempora, pval_threshold = pw.threshold)

print("PlotVaryingPWs")
PlotVaryingPWs(covid_tempora, clusterField, outprefix, pw.max)    
}


return(covid_tempora)
}


In [ ]:
makeStateCellnamesTP = function( scobj, thresh=2 )
{
  stateCellnamesTPnp = paste(unlist(
    lapply(str_split(scobj$orig.ident, ";"), function(x){return(str_replace(str_replace(x[1], "Gamma delta T cells", "gd T cells"),
     "Plasmacytoid dendritic cell", "pDC"))})
  ), scobj$mpoint2)

  print(table(stateCellnamesTPnp))

  scobj$stateCellnamesTP = stateCellnamesTPnp

  cellThreshold = thresh
  keepClusters = names(table(stateCellnamesTPnp)[table(stateCellnamesTPnp) > cellThreshold])
  removeClusters = table(stateCellnamesTPnp)[table(stateCellnamesTPnp) <= cellThreshold]
  print("removing clusters")
  print(removeClusters)

  scobj = subset(scobj,stateCellnamesTP %in% keepClusters)

  stateCellnamesTPnp = paste(unlist(
    lapply(str_split(scobj$orig.ident, ";"), function(x){return(str_replace(str_replace(x[1], "Gamma delta T cells", "gd T cells"),
     "Plasmacytoid dendritic cell", "pDC"))})
  ), scobj$mpoint2)

  stateCellnamesTPFactornp = factor(stateCellnamesTPnp)
  stateCellnamesTPNumericnp = as.numeric(stateCellnamesTPFactornp)
  scobj$stateCellnamesTP = stateCellnamesTPNumericnp

  return(list("seurat"=scobj, "factorObj"=stateCellnamesTPFactornp))
}


In [ ]:
tempora.bulk = list()
for (use.celltype in unique(obj.bulk$orig.ident))
{
    print(use.celltype)
    procObj.all = makeStateCellnamesTP(subset(obj.bulk, orig.ident == use.celltype & mpoint2 %in% c('Ctrl', 'Day4', 'Hosp.','Day11','Day60')), thresh=2)   
    tempora.all = analyseTempora(procObj.all$seurat, "stateCellnamesTP", levels(procObj.all$factorObj), "mpoint2", c('Ctrl', 'Day4', 'Hosp.','Day11','Day60'), paste("detests_time/",use.celltype,"_stateCellnamesTP_all",sep=""), pw.threshold=0.5, do.pw=F)
    
    tempora.bulk[[celltype]] = tempora.all
}

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5)

p=makeTrajectoryPlot(tempora.all)
p=save_plot(p, "detests_time/stateCellnamesTP_all_trajectory", fig.width=15, fig.height=7)

In [ ]:
PlotVaryingPWs(tempora.all, "stateCellnamesTP","detests_time/stateCellnamesTP_all")

In [ ]:
tempora.all = analyseTempora(procObj.all$seurat, "stateCellnamesTP", levels(procObj.all$factorObj), "mpoint", c('ctrl','tag4','tag11','tag60'), "detests_time/stateCellnamesTP_all", pw.threshold=0.5)

In [ ]:
tempora.all

In [ ]:
makeFuzzyPlot = function( resultsDF, selState = "sympt", list.name=NULL, logged=T, filterThresh=0, use.levels=list("-2"="DOWN", "-1"="down", "0"="No Reg.", "1"="up", "2"="UP"),map.labels=NULL, highlight=NULL, breaks=500, title="", filter.noreg=F)
{
  
statePattern = paste("\\.", selState, sep="")

ctr = resultsDF[, c("gene", grep(statePattern, grep("^bins.", colnames(resultsDF), value = T), value = T))]
#ctr$bins.0ctrl.tp0 = 0

ctr = ctr[, c("gene", sort(grep("^bins.", colnames(ctr), value = T)))]
rownames(ctr) = ctr$gene
inGenes = ctr$gene
    
ctr$gene = NULL

#print(ctr)


num2str = use.levels

for (bin_col in grep("^bins.", colnames(ctr), value = T))
{
  
  strVals = sapply(ctr[[bin_col]], function(x){num2str[[as.character(x)]]})
  strVals = as.factor(strVals)
  strVals = factor(strVals, levels=as.character(use.levels))
  
  ctr[[bin_col]] = strVals
  
  
}

#print(ctr)

if (logged)
{
  nfunc = function(x){return(log(x))}
  nfuncState = paste("n > ",filterThresh,"; log(n); ", sep="")
  
  yLabText = "Logged Gene Count"
} else {
  nfunc = function(x){return(x)}
  nfuncState = paste("n > ",filterThresh,"; ", sep="")
  
  yLabText = "Gene Count"
}

ctr[["highlighted"]] = 0
    
if (is.null(highlight))
{
  
  ctrFeature = ctr %>%
  dplyr::group_by(!!!syms(c(sort(grep("^bins.", colnames(ctr), value = T)), "highlighted"))) %>%
  dplyr::summarise(
    n = nfunc(n())  ) %>% dplyr::filter(n>nfunc(filterThresh)) %>% dplyr::arrange(desc(n))

    print(ctrFeature)
} else {
  
  
  ctr[highlight, c("highlighted")] = 1
  
  ctrFeature = ctr %>%
  dplyr::group_by(!!!syms(c(sort(grep("^bins.", colnames(ctr), value = T)), "highlighted"))) %>%
  dplyr::summarise(
    n = nfunc(n())  ) %>% dplyr::filter(highlighted==1 | n>nfunc(filterThresh)) %>% dplyr::arrange(desc(-highlighted,n))

}
    
fstate = "" 
if (filter.noreg)
{

ctrSmall = ctrFeature[grep("^bins.", colnames(ctr), value = T)]
ctrUse = rowSums(ctrSmall == "Unexpr.") != length(grep("^bins.", colnames(ctr), value = T))   
ctrFeature = ctrFeature[ctrUse | (ctrFeature$highlighted==1),]
    
    fstate = " filtered all Unexpr."
}
    

allgroups = paste("group", 1:nrow(ctrFeature), sep="")
allgroups = as.factor(allgroups)
allgroups = factor(allgroups, levels=paste("group", 1:nrow(ctrFeature), sep=""))

ctrFeature$group = allgroups
    
if (!is.null(list.name))
{
grpgenes = list()
binCols = sort(grep("^bins.", colnames(ctrFeature), value = T))
for (i in 1:nrow(ctrFeature)) {

    
    grpVec = as.vector(ctrFeature[i, binCols])
    grp = as.character(ctrFeature[i, "group"])
    
    for (j in 1:nrow(ctr))
    {
        rVec = as.vector(ctr[j,binCols])
        
        #print(sum(rVec == grpVec))
        if (sum(rVec == grpVec) == length(binCols))
        {
            #print(paste(grp, j, inGenes[j]))
            #print()
            grpgenes[[grp]] = c(grpgenes[[grp]], inGenes[j])
        }
    }
}
    

    resgrps = list()
    fname = paste(list.name, ".txt", sep="")
    print(fname)
        write(paste(binCols, collapse=", "), fname)

for (grp in names(grpgenes))
{
    
    subdf = ctrFeature[ctrFeature$group == paste("group", grp, sep=""), binCols]
    subdf = sapply(subdf[, binCols], as.character)
    dfvec = as.vector(subdf)
    resgrps[[grp]] = as.character(paste(paste("group", grp, sep=""), paste(dfvec, collapse=", "), paste(grpgenes[[grp]], collapse=", ")))

    write(paste(resgrps[[grp]]), fname, append=TRUE)

    
}
   
    }

aesMappings = list("y"="n")
i=1
    
use.labels = c()
if (!is.null(map.labels))
{
    for (axisCol in names(map.labels))
    {
      axisName = paste("axis", as.character(i), sep="")
      aesMappings[[axisName]] = axisCol
        
        use.labels = c(use.labels, map.labels[axisCol])
      i=i+1
    }
} else {
    for (axisCol in sort(grep("^bins.", colnames(ctr), value = T)))
    {
      axisName = paste("axis", as.character(i), sep="")
      aesMappings[[axisName]] = axisCol
        
        use.labels = c(use.labels, axisCol)
      i=i+1
    }
}
    


#print(aesMappings)
print(create_aes(aesMappings))
print(nrow(ctrFeature))

getPalette = colorRampPalette(brewer.pal(10, "Set3"))

colorValues = getPalette(nrow(ctrFeature))
colorValues[ctrFeature$highlighted == 1] = "#000000" #"#FF0000"

ctrFeature$highcolor = colorValues
#print(ctrFeature)

nLabels = length(grep("^bins.", colnames(ctrFeature), value = T))

geneCount = sum(ctrFeature$n)

if (!is.null(highlight))
{
      print(ctrFeature[ctrFeature$highlighted == 1,])

}
    
p=ggplot(as.data.frame(ctrFeature), create_aes(aesMappings)) +
  geom_alluvium(aes(fill = group, colour = group), width = 0, knot.pos = 0, reverse = FALSE, curve_type="quintic") +
  scale_fill_manual(values = colorValues)+
  scale_color_manual(values = colorValues)+
  guides(colour=FALSE,fill=FALSE) +
  geom_stratum(width = 1/16, reverse = FALSE) +
  geom_text(stat = "stratum", aes(label = after_stat(stratum)), reverse = FALSE, angle=90) +
  scale_x_continuous(breaks = 1:nLabels, labels = use.labels)+
  scale_y_continuous(limits = c(0, geneCount), breaks = seq(0, geneCount, by = breaks))+
  labs(
    title=paste(title, " Binned Expressions per TimePoint (", nfuncState, selState, fstate, ")", sep="")
  )+
  xlab("Timepoints")+
  ylab(yLabText)+
  theme(
    panel.background = element_blank(),
    axis.text.y = element_text(face="bold", size=14),
    axis.title.y = element_text(face="bold", size=24),
    axis.text.x = element_text(face="bold", size=14),
    axis.title.x = element_text(face="bold", size=24),
  )
    
    
return(p)
}


In [ ]:
library(dplyr)
library(ggpubr)
library(RColorBrewer)
library(ggalluvial)

In [ ]:
fuzzy.expr.all = prepareFuzzyDataExpr(geneExpr, "All", use.quantiles = c(0.75,0.9, 0.95))
fuzzy.expr.all$highlighted = 0

In [ ]:
fuzzy.expr.all.isg = fuzzy.expr.all[fuzzy.expr.all$gene %in% interferonGenes,]
makeFuzzyPlot(fuzzy.expr.all.isg, "", title=subsetName, logged=F, filterThresh = 0, use.levels=list("0"="Unexpr.", "1"="high", "2"="High", "3"="HIGH"), map.labels = list("bins.Ctrl"="Ctrl", "bins.Day4"="Day 4", "bins.Hosp."="Hosp.", "bins.Day11"="Day 11", "bins.Day60"="Day 60"), breaks=5)

In [ ]:
for (subsetName in names(geneExpr))
{

    fuzzy.expr.all = prepareFuzzyDataExpr(geneExpr, subsetName, use.quantiles = c(0.75,0.9, 0.95))
    fuzzy.expr.all.isg = fuzzy.expr.all[fuzzy.expr.all$gene %in% interferonGenes,]
    print(subsetName)
    fuzzyPrint = fuzzy.expr.all.isg[, c("gene", sort(grep("^bins", colnames(fuzzy.expr.all.isg), value=T)))]
    print(data.frame(gene=fuzzyPrint$gene, Day4 = fuzzyPrint$bins.Day4, Hosp=fuzzyPrint$bins.Hosp., Diff=fuzzyPrint$bins.Day4-fuzzyPrint$bins.Hosp. ))
}

In [ ]:
for (subsetName in names(geneExpr))
{

    fuzzy.expr.all = prepareFuzzyDataExpr(geneExpr, subsetName, use.quantiles = c(0.75,0.9, 0.95))

    pname = paste("detests_time/", "fuzzy_all_", subsetName, sep="")
    p=makeFuzzyPlot(fuzzy.expr.all, "", title=subsetName,filter.noreg=T, logged=F, highlight = interferonGenes, filterThresh = 10, use.levels=list("0"="Unexpr.", "1"="high", "2"="High", "3"="HIGH"), map.labels = list("bins.Ctrl"="Ctrl", "bins.Day4"="Day 4", "bins.Hosp."="Hosp.", "bins.Day11"="Day 11", "bins.Day60"="Day 60"), breaks=1000)
    save_plot(p, pname, 12, 12)

    fuzzy.expr.all.isg = fuzzy.expr.all[fuzzy.expr.all$gene %in% interferonGenes,]
    print(subsetName)
    
    fuzzyPrint = fuzzy.expr.all.isg[, c("gene", sort(grep("^bins", colnames(fuzzy.expr.all.isg), value=T)))]
    
    pname = paste("detests_time/", "fuzzy_isg_", subsetName, sep="")

    print(data.frame(gene=fuzzyPrint$gene, Day4 = fuzzyPrint$bins.Day4, Hosp=fuzzyPrint$bins.Hosp., Diff=fuzzyPrint$bins.Day4-fuzzyPrint$bins.Hosp. ))
    p=makeFuzzyPlot(fuzzy.expr.all.isg, "", title=subsetName, list.name=pname, logged=F, filterThresh = 0, use.levels=list("0"="Unexpr.", "1"="high", "2"="High", "3"="HIGH"), map.labels = list("bins.Ctrl"="Ctrl", "bins.Day4"="Day 4", "bins.Hosp."="Hosp.", "bins.Day11"="Day 11", "bins.Day60"="Day 60"), breaks=5)
    save_plot(p, pname, 12, 12)

}

In [ ]:
writeLines(as.vector(list("testn"="test", "testnn"="test2")), file("test"), sep="\n")

In [ ]:
fuzzy.expr.all.isg = fuzzy.expr.all[fuzzy.expr.all$gene %in% interferonGenes,]
makeFuzzyPlot(fuzzy.expr.all.isg, "", logged=F, filterThresh = 0, use.levels=list("0"="No Reg.", "1"="up", "2"="Up", "3"="UP"), map.labels = list("bins.Ctrl"="Ctrl", "bins.Day4"="Day 4", "bins.Hosp."="Hosp.", "bins.Day11"="Day 11", "bins.Day60"="Day 60"), breaks=1000)

In [ ]:
prepareFuzzyData = function(deObj, celltype, use.quantiles=c(0.25, 0.75))
{
    
    deDFs = deObj[[celltype]]

    celltype.logfcs = c()
    short.results = list()

    my_merge_outer <- function(df1, df2){                                # Create own merging function
      merge(df1, df2, by = "gene", all=TRUE)
    }

    for (condtp in names(deDFs))
    {
      for (tp in names(deDFs[[condtp]]))
      {
          celltype.logfcs = c(celltype.logfcs, deDFs[[condtp]][[tp]]$mean.asymptomatic)
      }
    }


    createBins = function(indf, col.fc, col.sig, bounds)
    {

      smalldf = indf[, c("gene", col.fc, col.sig)]
      colnames(smalldf) = c("gene", "fc", "sig")

      binFun = function(x)
      {

        fc = as.numeric(x[2])
        sig = as.numeric(x[3])

        #print(x)
        #print(bounds)

        if ((is.na(sig)) || is.nan(sig) ||(sig > 0.05) || is.na(fc) || is.nan(fc))
        {
          return(0)
        }

        curBin = 0

        if (fc > bounds[length(bounds)])
        {
          return(length(bounds))
        }

        for (i in 1:length(bounds))
        {
          #print(paste(i, curBin+i-1, fc, "<", bounds[i], fc < bounds[i]))
          if (fc < bounds[i])
          {
            return(curBin + i -1)
          }
        }

        print(x)
        return(10)
      }


      bins = apply(smalldf, 1, binFun)

      return(bins)

    }

    #testDF = data.frame(gene=c("gene1", "gene2", "gene3", "gene4", "gene5"),
    #                    log2FC = c(-1, -0.55, 0.1, 0.55, 1),
    #                    p_val_adj = c(0.01, 0.01, 0.01, 0.01,0.01)
    #)
    #createBins(testDF, "log2FC", "p_val_adj", c(-0.75, -0.5, 0.5, 0.75))


    quantileVec = as.numeric(quantile(abs(celltype.logfcs), probs = use.quantiles, na.rm=T))
    # using expression values!#quantileVec = c(-rev(quantileVec), quantileVec)
    print(paste("Quantile Vector"))
    print(quantileVec)

    for (condtp in names(deDFs))
    {

      condtp.results = list()
      for (tp in names(deDFs[[condtp]]))
      {

          celltype.logfcs = c(celltype.logfcs, deDFs[[condtp]][[tp]]$avg_log2FC)

          resDF = data.frame(gene=deDFs[[condtp]][[tp]]$gene)
          resDF[[paste("avg_log2FC", condtp, tp, sep=".")]] = deDFs[[condtp]][[tp]]$mean.asymptomatic
          resDF[[paste("p_val_adj", condtp, tp, sep=".")]] = 0
          resDF[[paste("pct", condtp, tp, sep=".")]] = deDFs[[condtp]][[tp]]$pct.2

          resDF[[paste("bins", condtp, tp, sep=".")]] = bins = createBins(resDF, paste("avg_log2FC", condtp, tp, sep="."),
                                                                          paste("p_val_adj", condtp, tp, sep="."), quantileVec)

          condtp.results[[tp]] = resDF
      }

      mergeDF = Reduce(my_merge_outer, condtp.results) 
      short.results[[condtp]] = mergeDF

    }

    celltype.results = Reduce(my_merge_outer, short.results)
    celltype.results



    for (binID in grep("^bins.", colnames(celltype.results), value = T))
    {
      celltype.results[[binID]][is.na(celltype.results[[binID]])] = 0

      for (condtp in names(short.results))
      {
          if (binID %in% colnames(short.results[[condtp]]))
          {
        short.results[[condtp]][[binID]][is.na(short.results[[condtp]][[binID]])] = 0    
          }

      }

    }

    return(celltype.results)
    
}

In [ ]:
interferonGenes = c("MT2A", "ISG15", "LY6E", "IFIT1", "IFIT2", "IFIT3", "IFITM1", "IFITM3", "IFI44L", "IFI6", "MX1", "IFI27",  "IFI44L", "RSAD2", "SIGLEC1", "IFIT1", "ISG15")

In [ ]:
fuzzy.monocytes = prepareFuzzyData(allResults, "Monocytes", use.quantiles=c(0.25, 0.5, 0.75))

In [ ]:
makeFuzzyPlot(fuzzy.monocytes, "asympt", logged=F, filterThresh = 10, use.levels=list("0"="No Reg.", "1"="up", "2"="Up", "3"="UP"))

In [ ]:
fuzzy.monocytes.isg = fuzzy.monocytes[fuzzy.monocytes$gene %in% interferonGenes,]
makeFuzzyPlot(fuzzy.monocytes.isg, selState = "asympt", filterThresh = 0, logged = F, use.levels=list("0"="No Reg.", "1"="up", "2"="Up", "3"="UP"))

In [ ]:
save.image("isg_plots_bulk.rdata")

In [ ]:
load("isg_plots_bulk.rdata")

In [ ]:
unique(subset(obj.bulk, orig.ident==cellt)$mpoint2)

In [ ]:
options(repr.plot.width=12, repr.plot.height=10)

for (cellt in unique(obj.bulk$orig.ident))
{
    ssobj = subset(obj.bulk, orig.ident==cellt)
    p=DotPlot(ssobj, features = unique(interferonGenes), group.by="mpoint2")+ggtitle(cellt)+coord_flip()
    plot(save_plot(p, paste("seurat/dotplot_isg_per_celltype", cellt), fig.width=12, fig.height=10))
}


In [ ]:
obj.mono = subset(obj.bulk, orig.ident=="Monocytes")

In [ ]:
obj.mono <- RunPCA(obj.mono, npcs = 30, verbose = FALSE)
obj.mono <- RunUMAP(obj.mono, reduction = "pca", dims = 1:30)
obj.mono <- FindNeighbors(obj.mono, reduction = "pca", dims = 1:30)
obj.mono <- FindClusters(obj.mono, resolution = 0.5)

In [ ]:
obj.plot = obj.mono
print(obj.plot)
p=DimPlot(obj.plot, reduction="pca", group.by="mpoint2", dims=c(1,2)) + theme(legend.position="bottom")+labs(title=element_blank(),suptitle=element_blank())
p$data$ct_time = obj.plot$ct_time
p$data$colors = obj.plot$colors
p = p+scale_color_manual(values = mpoint2Colors)

dens1 <- ggplot(p$data, aes(x = PC_1, fill = mpoint2))+scale_fill_manual(values = mpoint2Colors) + 
  geom_density(alpha = 0.4) + 
  theme_void() + 
  theme(legend.position = "none")

dens2 <- ggplot(p$data, aes(x = PC_2, fill = mpoint2))+scale_fill_manual(values = mpoint2Colors) + 
  geom_density(alpha = 0.4) + 
  theme_void() + 
  theme(legend.position = "none") + 
  coord_flip()

mp = dens1 + plot_spacer() + p + dens2 + plot_layout(ncol = 2, nrow = 2, widths = c(4, 1), heights = c(1, 4))

    if (is.null(title))
        {
        title=selCelltype
    }
    
title <- ggdraw() + draw_label("", fontface='bold')
p=cowplot::plot_grid(title, mp, ncol = 1, rel_heights = c(0.05, 1) )
p
#save_plot(p, paste("bulk_dimplots/mono_pca_1_2", sep=""), 8, 8)


In [ ]:
obj.mono = suppressWarnings(JackStraw(obj.mono))

In [ ]:
for (i in 1:15)
{
 pcaIgenes = PCASigGenes(obj.mono, pcs.use = c(i), pval.cut=0.01)    
    
 print(paste(i, length(pcaIgenes), length(intersect(pcaIgenes, interferonGenes))))
}

In [ ]:
obj.plot = obj.mono
print(obj.plot)
p=DimPlot(obj.plot, reduction="pca", group.by="mpoint2", dims=c(2,3)) + theme(legend.position="bottom")+labs(title=element_blank(),suptitle=element_blank())
p$data$ct_time = obj.plot$ct_time
p$data$colors = obj.plot$colors
p = p+scale_color_manual(values = mpoint2Colors)

dens1 <- ggplot(p$data, aes(x = PC_2, fill = mpoint2))+scale_fill_manual(values = mpoint2Colors) + 
  geom_density(alpha = 0.4) + 
  theme_void() + 
  theme(legend.position = "none")

dens2 <- ggplot(p$data, aes(x = PC_3, fill = mpoint2))+scale_fill_manual(values = mpoint2Colors) + 
  geom_density(alpha = 0.4) + 
  theme_void() + 
  theme(legend.position = "none") + 
  coord_flip()

mp = dens1 + plot_spacer() + p + dens2 + plot_layout(ncol = 2, nrow = 2, widths = c(4, 1), heights = c(1, 4))

    if (is.null(title))
        {
        title=selCelltype
    }
    
title <- ggdraw() + draw_label("", fontface='bold')
p=cowplot::plot_grid(title, mp, ncol = 1, rel_heights = c(0.05, 1) )
p
#save_plot(p, paste("bulk_dimplots/mono_pca_1_2", sep=""), 8, 8)
